# Start Code

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import json
import cv2
import numpy as np
import os

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create Torch Dataset

In [3]:
class KeypointsDataset(Dataset):

    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir

        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"self.img_dir/{item['id']}.png")
        h,w = img.shape[:2]

        # CV2 reads images in BGR format, convert to RGB
        img = cv2.cvTColor(img, cv2.COLOR_BGR2RGB)

        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        # Since we resized the image to 224x224, we need to scale the keypoints
        kps[0::2] *= (224.0/w)
        kps[1::2] *= (224.0/h)

        return img, kps

In [4]:
# Change directories to the location of the dataset
os.chdir("C:/Users/odiamant/Desktop/Work/my_stuff/vb_vision_project")

train_dataset = KeypointsDataset(
    img_dir= "training/tennis_court_det_dataset/data/images", 
    data_file= "training/tennis_court_det_dataset/data/data_train.json"
    )

valid_dataset = KeypointsDataset(
    img_dir= "training/tennis_court_det_dataset/data/images", 
    data_file= "training/tennis_court_det_dataset/data/data_val.json"
    )

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=True)

# Create Model for Keypoint Detection

In [5]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer with a new one that outputs 14*2 values (x,y for each keypoint)

c:\Users\odiamant\.conda\envs\cv_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\odiamant\.conda\envs\cv_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
model = model.to(device)

# Train model

In [7]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs= 20

for epoch in range(epochs):
    for i, (img,kps) in enumerate(train_loader): 
        imgs= imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        pred= model(imgs)
        loss = criterion(pred, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch: {epoch}, Batch: {i}, Loss: {loss.item()}")  

AttributeError: 'NoneType' object has no attribute 'shape'

# Save Model

In [ ]:
torch.save(model.state_dict(), "keypoints_model.pth")